In [27]:
from pynq import Overlay
from matplotlib import pyplot as plt
import scipy.io.wavfile as wavfile
from pynq import allocate
import pynq.lib.dma
import numpy as np
from numpy import fft
import random as rand
import operator
import scipy
import scipy.fftpack
from scipy.signal import argrelextrema

listOfFingerprints = dict()

In [28]:
overlay = Overlay('fft_bitstream/fft.bit')
dma = overlay.FFT.axi_dma_0

In [ ]:
def makeSongDict(filename):
    # ==============================================

    time_period = 0.1 # FFT time period (in seconds). Can comfortably process time frames from 0.05 seconds - 10 seconds

    # ==============================================
    


    fs_rate, signal_original = wavfile.read(filename)
    total_time = int(np.floor(len(signal_original)/fs_rate))
    sample_range = np.arange(0,total_time,time_period)
    total_samples = len(sample_range)

        #print ("Frequency sampling", fs_rate)
        #print ("total time: ", total_time)
        #print ("sample time period: ", time_period)
        #print ("total samples: ", total_samples)

    output_array = []
    for i in sample_range:

            #print ("Processing: %d / %d (%d%%)" % (i/time_period + 1, total_samples, (i/time_period + 1)*100/total_samples))

        sample_start = int(i*fs_rate)
        sample_end = int((i+time_period)*fs_rate)
        signal = signal_original[sample_start:sample_end]


        l_audio = len(signal.shape)
            #print ("Channels", l_audio)

        if l_audio == 2:
            signal = signal.sum(axis=1) / 2
        N = signal.shape[0]
            #print ("Complete Samplings N", N)

        secs = N / float(fs_rate)
            # print ("secs", secs)
        Ts = 1.0/fs_rate # sampling interval in time
            #print ("Timestep between samples Ts", Ts)

        t = scipy.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray

        FFT = abs(scipy.fft(signal))
        FFT_side = FFT[range(int(N/2))] # one side FFT range
        freqs = scipy.fftpack.fftfreq(signal.size, t[1]-t[0])
        fft_freqs = np.array(freqs)
        freqs_side = freqs[range(int(N/2))] # one side frequency range
        fft_freqs_side = np.array(freqs_side)

    fingerprint = abs(fft_freqs_side)
        
    listOfFingerprints[filename] = fingerprint

In [ ]:
makeSongDict('test.wav')

In [29]:
song1 = allocate(shape=(1024,),dtype=np.uint32)
song2 = allocate(shape=(1024,),dtype=np.uint32)
song3 = allocate(shape=(1024,),dtype=np.uint32)
song4 = allocate(shape=(1024,),dtype=np.uint32)
out_buffer = allocate(shape=(1024,), dtype = np.uint32)

song1[:] = [rand.randrange(0,10) for i in range(0, 1024)]
song2[:] = [rand.randrange(0,10) for i in range(0, 1024)]
song3[:] = [rand.randrange(0,10) for i in range(0, 1024)]
song4[:] = [rand.randrange(0,10) for i in range(0, 1024)]

songList = [song1, song2, song3, song4]
output_buffer_list = []

In [30]:
def run_kernel(songList):
    dma.sendchannel.transfer(songList)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [31]:
%%timeit
run_kernel(song1)
print(out_buffer)

[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293

[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293

[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293853254]
[      4671 4287496135    1179520 ...    1769513 4285857787 4293

In [32]:
FFT_side = out_buffer
# Reduce to 0-5000 Hz
bucket_size = 20
buckets = 16

FFT_side = FFT_side[0:bucket_size*buckets]

# Combine frequencies into buckets
FFT_side = np.array([int(sum(FFT_side[current: current+bucket_size])) for current in range(0, len(FFT_side), bucket_size)])

# FFT_side: Normalize (0-1)
max_value = max(FFT_side)
if (max_value != 0):
    FFT_side_norm = FFT_side / max_value

# Append to output array
output_array = FFT_side_norm

In [37]:
print(output_array)
output_array.tobytes()

[0.66648865 0.66676232 1.         0.60042251 0.93330302 0.7333942
 0.80023162 0.66685798 0.86686959 0.73369234 0.8664544  0.73342573
 0.86651038 0.46722085 0.66692921 0.46701836]


b'\x87\x95\xa5\x00\xe0S\xe5?\x029\x1f\xf0\x1dV\xe5?\x00\x00\x00\x00\x00\x00\xf0?\xfbL\x07C\xa96\xe3?\x82\xcd\x9cK\x9e\xdd\xed?u2\xb5\x1c\xf7w\xe7?\x10\xf5\x0cU\x7f\x9b\xe9?\xf0\xbb6\x8c\xe6V\xe5?{\xdcJLe\xbd\xeb?\x82\x016\\hz\xe7?\xe1E\xeb\x94\xfe\xb9\xeb?H\x8a\x05;9x\xe7?`.\x06\xf9s\xba\xeb?M\t\x0bG\xf2\xe6\xdd?\xaf\xfa\xd9\xed{W\xe5?%\x99\x88\xf5\xa0\xe3\xdd?'

In [39]:
output_array.nbytes

128

In [40]:
overlayCompare = Overlay('compare_bitstream/compare.bit')
overlayCompare?

compareDMA0 = overlayCompare.compare_dma.axi_dma_0
compareDMA1 = overlayCompare.compare_dma.axi_dma_1



In [41]:
generated_buffer = allocate(shape=(16,), dtype = np.uint32)
sample_buffer = allocate(shape=(16,), dtype = np.uint32)

generated_buffer[:] = output_array.data
#generated_buffer[:] = [rand.randrange(0,10) for i in range(0, 128)]
sample_buffer[:] = [rand.randrange(0,10) for i in range(0, 16)]

#sample_buffer = output_array
compare_output = allocate(shape=(16,), dtype = np.uint32)

In [42]:
def run_compare():
    compareDMA0.sendchannel.transfer(generated_buffer)
    compareDMA1.sendchannel.transfer(sample_buffer)
    compareDMA0.recvchannel.transfer(compare_output)
    compareDMA0.sendchannel.wait()
    compareDMA0.recvchannel.wait()

In [43]:
%%timeit
run_compare()
print(compare_output)

[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1

[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 1